In [248]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch.autograd import Variable

import time
import random
import numpy as np
import cvxpy as cp

from free_flyer.free_flyer import FreeFlyer
from free_flyer.utils import *
import pickle, os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
#load train/test data
prob = FreeFlyer() #use default config, pass different config file oth.
config_fn = './free_flyer/config/default.p'

config_file = open(config_fn,'rb')
dataset_name, _, _ = pickle.load(config_file); config_file.close()

relative_path = os.getcwd()
dataset_fn = relative_path + '/free_flyer/data/' + dataset_name

train_file = open(dataset_fn+'/train.p','rb')
# p_train, x_train, u_train, y_train, c_train, times_train = pickle.load(train_file)
train_data = pickle.load(train_file)
train_file.close()
x_train = train_data[1]
y_train = train_data[3]

test_file = open(dataset_fn+'/test.p','rb')
# p_test, x_test, u_test, y_test, c_test, times_test = pickle.load(test_file)
test_data = pickle.load(test_file)
p_test, x_test, u_test, y_test, c_test, times_test = test_data
test_file.close()

n_test = x_test.shape[0]

In [3]:
from solvers.mlopt_ff import MLOPT_FF

In [4]:
system = 'free_flyer'
prob_features = ['x0', 'obstacles_map']

mlopt_cnn = MLOPT_FF(system, prob, prob_features)

n_features = 4
mlopt_cnn.construct_strategies(n_features, train_data)
print(mlopt_cnn.n_strategies)

458


In [5]:
mlopt_cnn.setup_network()

fn_saved = 'None' # horizon of 11
mlopt_cnn.load_network(fn_saved)

mlopt_cnn.model

CNNet(
  (conv_activation): ReLU()
  (ff_activation): ReLU()
  (conv_layers): ModuleList(
    (0): Conv2d(3, 16, kernel_size=(2, 2), stride=(2, 2))
    (1): Conv2d(16, 16, kernel_size=(2, 2), stride=(2, 2))
    (2): Conv2d(16, 16, kernel_size=(2, 2), stride=(2, 2))
  )
  (ff_layers): ModuleList(
    (0): Linear(in_features=260, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=128, bias=True)
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): Linear(in_features=128, out_features=458, bias=True)
  )
)

In [6]:
mlopt_cnn.model

CNNet(
  (conv_activation): ReLU()
  (ff_activation): ReLU()
  (conv_layers): ModuleList(
    (0): Conv2d(3, 16, kernel_size=(2, 2), stride=(2, 2))
    (1): Conv2d(16, 16, kernel_size=(2, 2), stride=(2, 2))
    (2): Conv2d(16, 16, kernel_size=(2, 2), stride=(2, 2))
  )
  (ff_layers): ModuleList(
    (0): Linear(in_features=260, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=128, bias=True)
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): Linear(in_features=128, out_features=458, bias=True)
  )
)

In [15]:
def copy_shared_params(source, target, depth=3):
    last_layer_names = ['ff_layers.{}.weight'.format(depth), 'ff_layers.{}.bias'.format(depth)]
    source_params, target_params = source.named_parameters(), target.named_parameters()
    target_params_dict = dict(target_params)
    for name, param in source_params:
        if name in last_layer_names:
            continue
        if name in target_params_dict:
            target_params_dict[name].data.copy_(param.data)

In [283]:
def copy_all_but_last(target, source_data, depth=3):
    target_params = target.named_parameters()
    target_params_dict = dict(target_params)

    target_params_dict['conv_layers.0.weight'].data.copy_(source_data[0].data)
    target_params_dict['conv_layers.0.bias'].data.copy_(source_data[1].data)

    target_params_dict['conv_layers.1.weight'].data.copy_(source_data[2].data)
    target_params_dict['conv_layers.1.bias'].data.copy_(source_data[3].data)

    target_params_dict['conv_layers.2.weight'].data.copy_(source_data[4].data)
    target_params_dict['conv_layers.2.bias'].data.copy_(source_data[5].data)

    target_params_dict['ff_layers.0.weight'].data.copy_(source_data[6].data)
    target_params_dict['ff_layers.0.bias'].data.copy_(source_data[7].data)

    target_params_dict['ff_layers.1.weight'].data.copy_(source_data[8].data)
    target_params_dict['ff_layers.1.bias'].data.copy_(source_data[9].data)

    target_params_dict['ff_layers.2.weight'].data.copy_(source_data[10].data)
    target_params_dict['ff_layers.2.bias'].data.copy_(source_data[11].data)

def copy_last(target, source_data, depth=3):
    target_params = target.named_parameters()
    target_params_dict = dict(target_params)

    target_params_dict['ff_layers.3.weight'].data.copy_(source_data[12].data)
    target_params_dict['ff_layers.3.bias'].data.copy_(source_data[13].data)

In [189]:
from copy import deepcopy

feas_model = deepcopy(mlopt_cnn.model)
mlopt_model_copy = deepcopy(mlopt_cnn.model)

copy_shared_params(mlopt_model_copy, feas_model)

In [72]:
# num_train = train_data[0]['x0'].shape[0]

# np.random.seed(12)
# idxs = np.random.randint(0,num_train)

# inputs = torch.zeros((mlopt_cnn.problem.n_obs,mlopt_cnn.n_y))
# inputs = Variable(torch.from_numpy(mlopt_cnn.features[idxs,:])).float().to(device=mlopt_cnn.device)
# outputs = Variable(torch.from_numpy(mlopt_cnn.labels[idxs,:])).long().to(device=mlopt_cnn.device)

# copy_shared_params(feas_model, mlopt_model_copy)
# if 'obstacles_map' in prob_features:
#     params = train_data[0]
#     X_cnn = np.zeros((len(idxs), 3, mlopt_cnn.problem.H, mlopt_cnn.problem.W))
#     for idx_ii, idx_val in enumerate(idx):
#         prob_params = {}
#         for k in params:
#             prob_params[k] = params[k][mlopt_cnn.cnn_features_idx[idx_val][0]]
#         X_cnn[idx_ii] = mlopt_cnn.problem.construct_cnn_features(prob_params, \
#                         mlopt_cnn.prob_features, \
#                         ii_obs=mlopt_cnn.cnn_features_idx[idx_val][1])
#         cnn_inputs = Variable(torch.from_numpy(X_cnn)).float().to(device=mlopt_cnn.device)

# scores = mlopt_model_copy(cnn_inputs, inputs).detach().cpu().numpy()
# class_labels = np.argmax(scores, axis=1)

# y_guesses = np.zeros((len(idxs), mlopt_cnn.n_y))

# for ii,cl in enumerate(class_labels):
#     idx = np.where(mlopt_cnn.labels[:,0] == cl)[0][0]
#     y_guesses[ii,:] = mlopt_cnn.labels[idx,1:]

In [211]:
meta_lr = 1e-3
update_lr = 0.01

In [242]:
num_train = train_data[0]['x0'].shape[0]

np.random.seed(12)

loss = torch.zeros(1)

for _ in range(10):
    idx = np.random.randint(0,num_train)

#     copy_shared_params(feas_model, mlopt_model_copy)
    if 'obstacles_map' in prob_features:
        params = train_data[0]

        prob_params = {}
        for k in params:
            prob_params[k] = params[k][mlopt_cnn.cnn_features_idx[idx_val][0]]

        ff_inputs = torch.from_numpy(mlopt_cnn.problem.construct_features(prob_params, mlopt_cnn.prob_features))
        ff_inputs = Variable(ff.repeat(mlopt_cnn.problem.n_obs,1)).float().to(device=mlopt_cnn.device)

        X_cnn = np.zeros((mlopt_cnn.problem.n_obs, 3, mlopt_cnn.problem.H, mlopt_cnn.problem.W))
        for ii_obs in range(mlopt_cnn.problem.n_obs):
            X_cnn[ii_obs] = mlopt_cnn.problem.construct_cnn_features(prob_params, \
                            mlopt_cnn.prob_features, \
                            ii_obs=ii_obs)
        cnn_inputs = Variable(torch.from_numpy(X_cnn)).float().to(device=mlopt_cnn.device)

    scores = mlopt_model_copy(cnn_inputs, ff_inputs).detach().cpu().numpy()
    class_labels = np.argmax(scores, axis=1)

    y_guesses = np.zeros((mlopt_cnn.problem.n_obs, mlopt_cnn.n_y))
    y_guess = np.zeros((4*mlopt_cnn.problem.n_obs, mlopt_cnn.problem.N-1))

    for ii,cl in enumerate(class_labels):
        idx = np.where(mlopt_cnn.labels[:,0] == cl)[0][0]
        y_obs = mlopt_cnn.labels[idx,1:]
        y_guesses[ii,:] = y_obs
        y_guess[4*ii:4*(ii+1)] = np.reshape(y_obs, (4, mlopt_cnn.problem.N-1))

    prob_success = mlopt_cnn.problem.solve_pinned(prob_params, y_guess, solver=cp.MOSEK)[0]

    margin = 10.
    losses = torch.zeros(8,1)

    feas_scores = feas_model(cnn_inputs, ff_inputs)
    for ii_obs in range(mlopt_cnn.problem.n_obs):
        losses[ii_obs] = feas_scores[ii_obs,class_labels[ii_obs]]

#     print('worked' if prob_success else 'failed')
    if prob_success:
        loss += torch.relu(margin - torch.sum(losses))
    else:
        loss += torch.relu(margin + torch.sum(losses))

grad = torch.autograd.grad(loss, feas_model.parameters())
fast_weights = list(map(lambda p: p[1] - update_lr * p[0], zip(grad, feas_model.parameters())))

copy_all_but_last(mlopt_model_copy, fast_weights)

In [ ]:
# grab training params
BATCH_SIZE = mlopt_cnn.training_params['BATCH_SIZE']
TRAINING_ITERATIONS = mlopt_cnn.training_params['TRAINING_ITERATIONS']
TRAINING_ITERATIONS = 1
BATCH_SIZE = mlopt_cnn.training_params['BATCH_SIZE']
BATCH_SIZE = 256
CHECKPOINT_AFTER = mlopt_cnn.training_params['CHECKPOINT_AFTER']
SAVEPOINT_AFTER = mlopt_cnn.training_params['SAVEPOINT_AFTER']
TEST_BATCH_SIZE = mlopt_cnn.training_params['TEST_BATCH_SIZE']

NUM_META_PROBLEMS = np.maximum(int(BATCH_SIZE / mlopt_cnn.problem.n_obs), 50)

model = mlopt_model_copy
model.to(device=mlopt_cnn.device)

X = mlopt_cnn.features[:mlopt_cnn.problem.n_obs*mlopt_cnn.num_train]
X_cnn = None
if 'obstacles_map' in mlopt_cnn.prob_features:
# X_cnn = self.cnn_features[:self.problem.n_obs*self.num_train]
# TODO(acauligi)
    params = train_data[0]
    X_cnn = np.zeros((BATCH_SIZE, 3,mlopt_cnn.problem.H,mlopt_cnn.problem.W))
    Y = mlopt_cnn.labels[:mlopt_cnn.problem.n_obs*mlopt_cnn.num_train,0]

training_loss = torch.nn.CrossEntropyLoss()
meta_opt = torch.optim.Adam(model.parameters(), lr=meta_lr, weight_decay=0.00001)


itr = 1
for epoch in range(TRAINING_ITERATIONS):  # loop over the dataset multiple times
    t0 = time.time()
    running_loss = 0.0
    rand_idx = list(np.arange(0,X.shape[0]-1))
    random.shuffle(rand_idx)

    # Sample all data points
    indices = [rand_idx[ii * BATCH_SIZE:(ii + 1) * BATCH_SIZE] for ii in range((len(rand_idx) + BATCH_SIZE - 1) // BATCH_SIZE)]

    for ii,idx in enumerate(indices):
        # zero the parameter gradients
        meta_opt.zero_grad()

        inner_loss = torch.zeros(1)
        for _ in range(NUM_META_PROBLEMS):
            idx_val = np.random.randint(0,num_train)

        #     copy_shared_params(feas_model, mlopt_model_copy)
            if 'obstacles_map' in prob_features:
                params = train_data[0]

                prob_params = {}
                for k in params:
                    prob_params[k] = params[k][mlopt_cnn.cnn_features_idx[idx_val][0]]

                ff_inputs = torch.from_numpy(mlopt_cnn.problem.construct_features(prob_params, mlopt_cnn.prob_features))
                ff_inputs = Variable(ff.repeat(mlopt_cnn.problem.n_obs,1)).float().to(device=mlopt_cnn.device)

                X_cnn = np.zeros((mlopt_cnn.problem.n_obs, 3, mlopt_cnn.problem.H, mlopt_cnn.problem.W))
                for ii_obs in range(mlopt_cnn.problem.n_obs):
                    X_cnn[ii_obs] = mlopt_cnn.problem.construct_cnn_features(prob_params, \
                                    mlopt_cnn.prob_features, \
                                    ii_obs=ii_obs)
                cnn_inputs = Variable(torch.from_numpy(X_cnn)).float().to(device=mlopt_cnn.device)

            scores = mlopt_model_copy(cnn_inputs, ff_inputs).detach().cpu().numpy()
            class_labels = np.argmax(scores, axis=1)

            y_guesses = np.zeros((mlopt_cnn.problem.n_obs, mlopt_cnn.n_y))
            y_guess = np.zeros((4*mlopt_cnn.problem.n_obs, mlopt_cnn.problem.N-1))

            for ii,cl in enumerate(class_labels):
                cl_idx = np.where(mlopt_cnn.labels[:,0] == cl)[0][0]
                y_obs = mlopt_cnn.labels[cl_idx,1:]
                y_guesses[ii,:] = y_obs
                y_guess[4*ii:4*(ii+1)] = np.reshape(y_obs, (4, mlopt_cnn.problem.N-1))

            prob_success = mlopt_cnn.problem.solve_pinned(prob_params, y_guess, solver=cp.MOSEK)[0]

            margin = 10.
            losses = torch.zeros(8,1)

            feas_scores = feas_model(cnn_inputs, ff_inputs)
            for ii_obs in range(mlopt_cnn.problem.n_obs):
                losses[ii_obs] = feas_scores[ii_obs,class_labels[ii_obs]]

        #     print('worked' if prob_success else 'failed')
            if prob_success:
                inner_loss += torch.relu(margin - torch.sum(losses))
            else:
                inner_loss += torch.relu(margin + torch.sum(losses))

        grad = torch.autograd.grad(inner_loss, feas_model.parameters())
        fast_weights = list(map(lambda p: p[1] - update_lr * p[0], zip(grad, feas_model.parameters())))

        copy_all_but_last(mlopt_model_copy, fast_weights)

        ff_inputs = Variable(torch.from_numpy(X[idx,:])).float().to(device=mlopt_cnn.device)
        labels = Variable(torch.from_numpy(Y[idx])).long().to(device=mlopt_cnn.device)

        # forward + backward + optimize
        outputs = None
        if 'obstacles_map' in mlopt_cnn.prob_features:
            X_cnn = np.zeros((len(idx), 3,mlopt_cnn.problem.H,mlopt_cnn.problem.W))
            for idx_ii, idx_val in enumerate(idx):
                prob_params = {}
                for k in params:
                    prob_params[k] = params[k][mlopt_cnn.cnn_features_idx[idx_val][0]]
                X_cnn[idx_ii] = mlopt_cnn.problem.construct_cnn_features(prob_params, mlopt_cnn.prob_features, ii_obs=mlopt_cnn.cnn_features_idx[idx_val][1])
            cnn_inputs = Variable(torch.from_numpy(X_cnn)).float().to(device=mlopt_cnn.device)
            outputs = model(cnn_inputs, ff_inputs)
        else:
            outputs = model(ff_inputs)

        loss = training_loss(outputs, labels).float().to(device=mlopt_cnn.device)
        class_guesses = torch.argmax(outputs,1)
        accuracy = torch.mean(torch.eq(class_guesses,labels).float())
        loss.backward()
        #torch.nn.utils.clip_grad_norm(model.parameters(),0.1)
        meta_opt.step()

        copy_last(feas_model, [fw.detach() for fw in fast_weights])
        copy_shared_params(feas_model, model)

        itr += 1
        if ii % 100 == 0:
            print(ii)